In [ ]:
!pip install jupyter-dash

# Importamos nuestros paquetes

In [ ]:

from jupyter_dash import JupyterDash # Versión de dash para notebooks
import dash_core_components as dcc # Componentes HTML preconstruidos para dashboards
import dash_html_components as html # Componentes HTML nativos
from dash.dependencies import Input, Output # Clases Input y Output
import plotly.express as px # Generar gráficas e importar datasets con Plotly

import pandas as pd # Recolección y manipulación de datos

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  after removing the cwd from sys.path.


# Dash App interactiva simple

Nuestro primer acercamiento a los callbacks.

In [ ]:
# Importamos una hoja de estilos externa
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
#external_stylesheets = ['style/mihoja.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.H6("Change the value in the text box to see callbacks in action!"),
    html.Div(["Input: ",
              dcc.Input(id='my-input', value='initial value', type='text')]),
    html.Br(),
    html.Div(id='my-output'),

], style={'background-color': '#FFF0B1'})

@app.callback(
    Output(component_id='my-output', component_property='children'),
    Input(component_id='my-input', component_property='value')
)
def update_output_div(input_value):
    return 'Output: {}'.format(input_value)


if __name__ == '__main__':
    app.run_server(debug=True, mode='inline')

<IPython.core.display.Javascript object>

|Analicemos este ejemplo:

- En primer lugar, observe este código en el bloque anterior: `@app.callback`. El nombre una función, precedido por un `@` se llama "decorador", entonces ¿qué es un decorador? 
Muy en resumen, ***los decoradores envuelven una función, modificando su comportamiento***. Te dejamos [aquí](https://realpython.com/primer-on-python-decorators/#simple-decorators) un artículo para conocer más acerca de ellos.
- Las "entradas" (Input) y "salidas" (Output) de la interfaz de nuestra aplicación se declaran como los argumentos del decorador @app.callback.
[Obtén más información sobre cómo usar el decorador `@app.callback`.](https://dash.plotly.com/basic-callbacks)
- En Dash, las entradas y salidas de nuestra aplicación son simplemente las propiedades de un componente en particular. En este ejemplo, nuestra entrada es la propiedad "value" del componente que tiene el ID "my-input". Nuestra salida es la propiedad "children" del componente con el ID "my-output".
- Siempre que cambia un input (entrada), la función que envuelve el decorador de callback será llamada automáticamente. Dash proporciona a la función el nuevo valor de la propiedad de entrada como argumento de entrada y Dash actualiza la propiedad del componente de salida con lo que devolvió la función.
- Las palabras clave `component_id` y `component_property` son opcionales (solo hay dos argumentos para cada uno de esos objetos).
- No confunda el objeto `dash.dependencies.Input` y el objeto `dash_core_components.Input`. El primero solo se usa para las entradas de un callback, el segundo es un componente HTML completo similar a la [tag input](https://www.w3schools.com/tags/tag_input.asp).
- Observe cómo no establecemos un valor para la propiedad secundaria del componente `my-output` en el diseño. ***Cuando se inicia la aplicación Dash, automáticamente llama a todos los callbacks con los valores iniciales de los componentes de entrada para completar el estado inicial de los componentes de salida***. En este ejemplo, si especificaste algo como `html.Div (id = 'my-output', children = 'Hello world')`, se sobrescribirá cuando se inicie la aplicación. Es como programar con Microsoft Excel: **cada vez que cambia una celda de entrada, todas las celdas que dependen de esa celda se actualizarán automáticamente**. Esto se denomina "Programación reactiva".

Con la interactividad de Dash, podemos actualizar dinámicamente cualquiera de esas propiedades a través de un callback. Con frecuencia, actualizaremos los hijos de un componente para mostrar texto nuevo o actualizaremos la gráfica construida con `dcc.Graph` para mostrar nuevos datos, pero también podríamos actualizar el estilo de un componente o incluso las opciones disponibles de un componente `dcc.Dropdown` (como veremos más adelante).

# Dash App con slider

En este ejemplo veremos cómo el componente `dcc.Slider` actualiza al elemento `dcc.Graph`

## Obteniendo los datos de expectativa de vida de Gapminder

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv')

In [ ]:
df.info

<bound method DataFrame.info of           country  year         pop continent  lifeExp   gdpPercap
0     Afghanistan  1952   8425333.0      Asia   28.801  779.445314
1     Afghanistan  1957   9240934.0      Asia   30.332  820.853030
2     Afghanistan  1962  10267083.0      Asia   31.997  853.100710
3     Afghanistan  1967  11537966.0      Asia   34.020  836.197138
4     Afghanistan  1972  13079460.0      Asia   36.088  739.981106
...           ...   ...         ...       ...      ...         ...
1699     Zimbabwe  1987   9216418.0    Africa   62.351  706.157306
1700     Zimbabwe  1992  10704340.0    Africa   60.377  693.420786
1701     Zimbabwe  1997  11404948.0    Africa   46.809  792.449960
1702     Zimbabwe  2002  11926563.0    Africa   39.989  672.038623
1703     Zimbabwe  2007  12311143.0    Africa   43.487  469.709298

[1704 rows x 6 columns]>

## Elementos HTML

Recuerda que Dash admite el uso de elementos HTML para realizar el dashboard (de hecho, gracias a eso puede existir el dashboard). Entonces, considera el siguiente código y nota que con el paquete `dash_html_components.html` estamos agregando una etiqueta `div`, un heading 6 (`H6`) y una etiqueta `br`

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
# external_stylesheets = ['style/mihoja.css']

#app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    dcc.Graph(id='graph-with-slider'),
    dcc.Slider(
        id='year-slider',
        min=df['year'].min(),
        max=df['year'].max(),
        value=df['year'].min(),
        marks={str(year): str(year) for year in df['year'].unique()},
        step=None
    )
], style={'background-color': '#242a44'})


## Utilizando callbacks para actualizar la gráfica

In [ ]:
@app.callback(
    Output('graph-with-slider', 'figure'),
    Input('year-slider', 'value'))
def update_figure(selected_year):
    filtered_df = df[df.year == selected_year]

    fig = px.scatter(filtered_df, x="gdpPercap", y="lifeExp",
                     size="pop", color="continent", hover_name="country",
                     log_x=True, size_max=55)

    fig.update_layout(transition_duration=500)

    return fig


if __name__ == '__main__':
    app.run_server(debug=True, mode='inline')

<IPython.core.display.Javascript object>

En este ejemplo, la propiedad `"value"` del `Slider` es la entrada de la aplicación y la salida de la aplicación es la propiedad `"figure"` del gráfico. Siempre que cambia el valor del Slider, Dash llama al callback `update_figure` con el nuevo valor. *La función filtra el dataframe con el valor actual de `value`, construye un nuevo objeto de `figure` y lo devuelve a la aplicación Dash.*

Hay algunos patrones agradables en este ejemplo:

1. Estamos usando la biblioteca Pandas para ***importar y filtrar*** conjuntos de datos en la memoria.
2. Cargamos dataframe al inicio de la aplicación: `df = pd.read_csv ('...')`. ***Este dataframe (`df`) está en el estado global de la aplicación y se puede leer dentro de los callbacks***.
3. Cargar datos en la memoria puede resultar caro. Al cargar los datos de consulta al inicio de la aplicación en lugar de dentro de las funciones de devolución de llamada, nos aseguramos de que esta operación solo se realice cuando se inicie el servidor de la aplicación. Cuando un usuario visita la aplicación o interactúa con la aplicación, esos datos (el df) ya están en la memoria. Si es posible, la inicialización costosa (como descargar o consultar datos) debe realizarse en el alcance global de la aplicación en lugar de dentro de los callbacks.
4. El callback no modifica los datos originales, solo crea copias del marco de datos filtrando a través de filtros pandas. Esto es importante: sus callbacks nunca deben mutar variables fuera de su alcance. Si sus callbacks modifican el estado global, entonces la sesión de un usuario podría afectar la sesión del siguiente usuario y cuando la aplicación se implemente en varios procesos o subprocesos, esas modificaciones no se compartirán entre sesiones.
5. Estamos activando las transiciones con `layout.transition` para dar una idea de cómo evoluciona el conjunto de datos con el tiempo: las transiciones permiten que el gráfico se actualice de un estado al siguiente sin problemas, como si estuviera animado.

# Dash App con múltiples entradas

**texto en negrita**## Preparamos el terreno con el layout y los datos

In [ ]:
df = pd.read_csv('https://plotly.github.io/datasets/country_indicators.csv')

df.head()

,Country Name,Indicator Name,Year,Value
0,Arab World,"Agriculture, value added (% of GDP)",1962,NaN
1,Arab World,CO2 emissions (metric tons per capita),1962,0.760996
2,Arab World,Domestic credit provided by financial sector (...,1962,18.168690
3,Arab World,Electric power consumption (kWh per capita),1962,NaN
4,Arab World,Energy use (kg of oil equivalent per capita),1962,NaN


In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
# external_stylesheets = ['style/mihoja.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

df = pd.read_csv('https://plotly.github.io/datasets/country_indicators.csv')

available_indicators = df['Indicator Name'].unique()

app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                id='xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Fertility rate, total (births per woman)'
            ),
            dcc.RadioItems(
                id='xaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ],
        style={'width': '48%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                id='yaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Life expectancy at birth, total (years)'
            ),
            dcc.RadioItems(
                id='yaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ],style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
    ]),

    dcc.Graph(id='indicator-graphic'),

    dcc.Slider(
        id='year--slider',
        min=df['Year'].min(),
        max=df['Year'].max(),
        value=df['Year'].max(),
        marks={str(year): str(year) for year in df['Year'].unique()},
        step=None
    )
], style={'background-color': '#FFF0B1'})

#if __name__ == '__main__':
#    app.run_server(debug=True, mode='inline')


## Hacemos el callback y la función para mantener actualizada nuestra gráfica

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']


@app.callback(
    Output('indicator-graphic', 'figure'),
    Input('xaxis-column', 'value'),
    Input('yaxis-column', 'value'),
    Input('xaxis-type', 'value'),
    Input('yaxis-type', 'value'),
    Input('year--slider', 'value'))
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    # Aquí filtramos por año
    dff = df[df['Year'] == int(year_value)]

    # Aquí hacemos el scatter
    fig = px.scatter(x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
                     y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
                     hover_name=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'])

    # Aquí se actualiza la gráfica con los datos filtrados
    fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')

    fig.update_xaxes(title=xaxis_column_name,
                     type='linear' if xaxis_type == 'Linear' else 'log')

    fig.update_yaxes(title=yaxis_column_name,
                     type='linear' if yaxis_type == 'Linear' else 'log')
    
    return fig


if __name__ == '__main__':
    app.run_server(debug=True, mode='inline')
    app.enable_dev_tools(
    dev_tools_ui=True,
    dev_tools_serve_dev_bundles=True,
    ) 

<IPython.core.display.Javascript object>

En este ejemplo, se llama a la función `update_graph` cada vez que cambia la propiedad value de los componentes Dropdown, Slider o RadioItems.

Los argumentos de entrada de la función `update_graph` son el valor nuevo o actual de cada una de las propiedades de entrada, en el orden en que se especificaron.

Aunque solo cambia una entrada a la vez (un usuario solo puede cambiar el valor de un menú desplegable en un momento dado), Dash recopila el estado actual de todas las propiedades de entrada especificadas y las pasa a su función automáticamente. Siempre se garantiza que sus callbacks pasarán al estado representativo de la aplicación.

# Dash App con múltiples salidas

Hasta ahora, todos los callbacks que hemos escrito solo actualizan una única propiedad de salida. También podemos actualizar varias a la vez: coloque todas las propiedades que desea actualizar como una lista en el decorador y devuelva esa cantidad de elementos en el callback.

In [ ]:

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']


app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    dcc.Input(
        id='num-multi',
        type='number',
        value=5
    ),
    html.Table([
        html.Tr([html.Td(['x', html.Sup(2)]), html.Td(id='square')]),
        html.Tr([html.Td(['x', html.Sup(3)]), html.Td(id='cube')]),
        html.Tr([html.Td([2, html.Sup('x')]), html.Td(id='twos')]),
        html.Tr([html.Td([3, html.Sup('x')]), html.Td(id='threes')]),
        html.Tr([html.Td(['x', html.Sup('x')]), html.Td(id='x^x')]),
    ]),
], style={'background-color': '#FFF0B1'})

@app.callback(
    Output('square', 'children'),
    Output('cube', 'children'),
    Output('twos', 'children'),
    Output('threes', 'children'),
    Output('x^x', 'children'),
    Input('num-multi', 'value'))
def callback_a(x):
    return x**2, x**3, 2**x, 3**x, x**x


if __name__ == '__main__':
    app.run_server(debug=True, mode='inline')

<IPython.core.display.Javascript object>

Una advertencia: no siempre es una buena idea combinar Salidas, incluso si puede:

- Si las salidas dependen de algunas entradas, pero no de todas, mantenerlas separadas puede evitar actualizaciones innecesarias.
- Si tienen las mismas entradas pero se realizan cálculos independientes con estas entradas, mantener los callbacks separados puede permitir que se ejecuten en paralelo.

# Dash App con callbacks encadenados

También se pueden encadenar salidas y entradas en un callback: la salida de un callback puede ser la entrada de otro callback.

Este patrón se puede utilizar para crear interfaces de usuario dinámicas en las que un componente de entrada actualiza las opciones disponibles del siguiente componente de entrada. He aquí un ejemplo sencillo:

In [ ]:
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

all_options = {
    'America': ['New York City', 'San Francisco', 'Cincinnati','Atlanta'],
    'Canada': [u'Montréal', 'Toronto', 'Ottawa']
}
app.layout = html.Div([
    dcc.RadioItems(
        id='countries-radio',
        options=[{'label': k, 'value': k} for k in all_options.keys()],
        value='America'
    ),

    html.Hr(),

    dcc.RadioItems(id='cities-radio'),

    html.Hr(),

    html.Div(id='display-selected-values')
], style={'background-color': '#FFF0B1'})


@app.callback(
    Output('cities-radio', 'options'),
    Input('countries-radio', 'value'))
def set_cities_options(selected_country):
  
    return [{'label': i, 'value': i} for i in all_options[selected_country]]


@app.callback(
    Output('cities-radio', 'value'),
    Input('cities-radio', 'options'))
def set_cities_value(available_options):
    return available_options[0]['value']


@app.callback(
    Output('display-selected-values', 'children'),
    Input('countries-radio', 'value'),
    Input('cities-radio', 'value'))
def set_display_children(selected_country, selected_city):
    return '{} is a city in {}'.format(
        selected_city, selected_country,
    )


if __name__ == '__main__':
    app.run_server(debug=True, mode='inline')

<IPython.core.display.Javascript object>